Dhall is a purely functional configuration language.

The official site [dhall-lang.org](https://dhall-lang.org) contains user documentation, while the [github repository](https://github.com/dhall-lang/dhall-lang/blob/master/standard/) contains a formal specification of the language, intended for developers.

Dhall programs look like this:

     let f = λ(x : Natural) → λ(y : Natural) → x + y + 2
     let id = λ(a : Type) → λ(x : a) → x
     in f 10 (id Natural 20)
       -- This is a complete Dhall program; it evaluates to 32 of type Natural.

Main features of Dhall:

- simple syntax, everything is a statically typed expression
- few built-in types and functions (`Natural`, `Bool`, `Text`, `Optional`, `List`, and some others)
- safety: no side effects, no mutation, no exceptions, no partial functions
- a very powerful type system: Dhall implements System F-omega but treats types similarly to values, has structurally typed records and unions, and basic support for dependent types
- restricted computations: no general recursion, no general string comparison
- file-based module system with safe imports

Main use case for Dhall is to represent at once an entire JSON or YAML configuration file and its data schema with some validations.

A Dhall script for a small configuration file could look like this:

     let Text/nonEmpty = ./text-utils/Text/nonEmpty.dhall
     let Natural/greaterThan = https://prelude.dhall-lang.org/Natural/greaterThan
     let username = env:DB_USER as Text
     let password = env:DB_PASS as Text
     let port : Natural = env:DB_PORT
     let _ = assert : Text/nonEmpty password
     let _ = assert : Text/nonEmpty username
     let _ = assert : Natural/greaterThan port 1024 === True
     in {
       db-host = "localhost:${port}"
       db-password = password
       db-username = username
     }

This script can be run via `dhall-json` or `dhall-yaml` utilities to generate JSON or YAML configuration files. Those files will not be generated if any validations fail (the `assert` expressions).

We will implement µDhall, a subset of Dhall that is just about large enough to run the first example program shown above.

We will not implement records, union types, or imports. Our goal is to learn how a functional language can be specified and implemented. Once we learn that, we will be able to add new features safely and consistently.

Here is an example µDhall program that uses all features we will support:

     let f = λ(x : Natural) → λ(x : Natural) → (123 + x) * x@1
     let id = λ(a : Type) → λ(x : a) → x
     let type_of_id = ∀(a : Type) → a → a
     let _ = id : type_of_id
     let _ = type_of_id : Type
     let _ = Type : Kind
     let Void = ∀(r : Type) → r
     let Unit = ∀(r : Type) → r 
     let Pair = λ(a : Type) → λ(b : Type) → ∀(r : Type) → (a → b → r) → r
     in f 10 (id Natural 20)

We will implement parsing, type-checking, and evaluation of µDhall programs.